## Evaluating OpenFold-3 Results with Biotite
In this notebook we will co-fold a protein-ligand complex with OpenFold-3 and use Biotite to calculate the RMSD between the co-folded structure and the corresponding x-ray structure from the PDB.  We begin by setting up the co-folding, which requires a protein sequence and the SMILES for a ligand. In this notebook we follow these steps.
- Download a structure from the PDB
- Extract the Ligand From the Protein Structure and Convert to SMILES
- Get the Protein Sequence from the Structure
- Perform Co-Folding
- Calculate the RMSD Between the X-ray and OF-3 ligands

### Install the necessary Python libraries

In [1]:
import sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    !uv pip install plinder biotite rdkit py3Dmol requests
    !uv pip install "numpy>2.3"

The next cell is a bit of a hack.  The plinder library installs an old version of numpy that creates problems for the other libraries.  To overcome this, we have to force a kernel restart.  When you execute the cell below for the first time, you'll see a message saying that your notebook crashed.  Don't despair, just run the cell below again and everything will work.   

In [2]:
if IN_COLAB:
  import numpy as np
  np_version = np.__version__
  if np_version < '2.3':
    exit()

Now we will import the necessary Python libraries.  You may see a few messages about `invalid escape sequence`.  Ignore those.

In [3]:
import biotite.structure.io.pdbx as pdbx  # read and write PDB and CIF files
import biotite.database.rcsb as rcsb      # download data from the RCSB PDB
import biotite.structure as struc         # structure handling
from biotite.interface import rdkit       # Biotite interface to the RDKit
from biotite.structure.io import pdb       # Biotite interface to the PDB
from rdkit import Chem                    # RDKit Chemistry library
from rdkit.Chem import rdMolAlign         # RMSD calculation
import zipfile                            # reading the zip file output by OpenFold-3
from pathlib import Path                  # renaming files
from plinder.core.structure.superimpose import superimpose_chain  #superimposing structures
import py3Dmol                            # 3D display of molecules
import numpy as np                        # Used to filter Biotite structures
import requests                           # downloading files
import io

### Download a structure from the PDB
We begin by downloading the CIF file for a protein-ligand complex from the PDB.  By default we're using `1BMK` a structure of p38$\alpha$ kinase.  To use a different protein structure, just change the `pdb_id` and `ligand_ccid` below.

In [4]:
pdb_id = '1bmk'
ligand_ccid = 'SB5'
file_path = rcsb.fetch(pdb_id, 'cif', target_path='.')

To ensure that we don't overwrite the file, we'll rename it.

In [5]:
path = Path(file_path)
path = path.rename("ref_"+path.name)

Read the downloaded structure using Biotite. The result is a Biotite `atom_array`.

In [6]:
ref_pdbx_file = pdbx.CIFFile.read(path)
ref_atom_array = pdbx.get_structure(ref_pdbx_file, model=1, include_bonds=True)

### Extract the Ligand From the Protein Structure and Convert to SMILES

Create a mask for the ligand residue in chain A and use that mask to create an `atom_array` with just the ligand

In [7]:
ref_ligand_mask = (ref_atom_array.res_name == ligand_ccid) & (ref_atom_array.chain_id == 'A')
ref_ligand_atom_array = ref_atom_array[ref_ligand_mask]

Convert the ligand `atom_array` to an `RDKitMolecule`

In [8]:
ligand_rd_mol = rdkit.to_mol(ref_ligand_atom_array)

Generate a SMILES from the RDKit molecule

In [9]:
ref_ligand_smiles = Chem.MolToSmiles(ligand_rd_mol)
ref_ligand_smiles

'Nc1nccc(c2c(c3ccc(F)cc3)ncn2CC2CC2)n1'

### Get the Protein Sequence from the Structure
Generate the protein sequence

In [10]:
chain_A_mask = ref_atom_array.chain_id == "A"
protein_mask = struc.filter_amino_acids(ref_atom_array)
protein = ref_atom_array[protein_mask & chain_A_mask]
sequences = struc.to_sequence(protein)
seq_1bmk_A = "".join(sequences[0][0])
seq_1bmk_A

'ERPTFYRQELNKTIWEVPERYQNLSPVGSGAYGSVCAAFDTKTGHRVAVKKLSRPFQSIIHAKRTYRELRLLKHMKHENVIGLLDVFTPARSLEEFNDVYLVTHLMGADLNNIVKCQKLTDDHVQFLIYQILRGLKYIHSADIIHRDLKPSNLAVNEDCELKILDFGLARHTDDEMTGYVATRWYRAPEIMLNWMHYNQTVDIWSVGCIMAELLTGRTLFPGTDHIDQLKLILRLVGTPGAELLKKISSESARNYIQSLAQMPKMNFANVFIGANPLAVDLLEKMLVLDSDKRITAAQALAHAYFAQYHDPDDEPVADPYDQSFESRDLLIDEWKSLTYDEVISFVPPPLD'

### Perform Co-Folding
Now that we've generated the ligand SMILES and the protein sequence, we can use [ApherisFold-Lite](https://try.fold.apheris.net/) to do the co-folding.  Under the hood, this method is using OpenFold-3 for the co-folding. To use ApherisFold-Lite, follow these steps.
1. Give the job a name in the `Name` box. Anything will do.
2. Paste the sequence into the `Sequence` box.
3. Click on `Add Chain`
4. Set the `Molecule Type` to **Ligand**
5. Paste the SMILES
6. Click `Submit`

The screenshot below shows an example.

<img src="https://raw.githubusercontent.com/PatWalters/practical_cheminformatics_tutorials/main/biotite/images/apheris_lite.png" alt="" width="800" />

Once the co-folding completes, navigate to the `Results` page. On the `Results` page, select "Download Raw Results".  I'll provide a pointer to the results generated from my run. You can substitute your file here.  Once we've read the zip file, we can use Python's `ZipFile` library to unpack it.

In [11]:
url = "https://raw.githubusercontent.com/PatWalters/datafiles/refs/heads/main/co-folding/job_He2COJAL7-results.zip"
filename = "job_He2COJAL7-results.zip"

# 'stream=True' is critical for large files
response = requests.get(url, stream=True)

if response.status_code == 200:
    with open(filename, 'wb') as f:
        # iter_content breaks the file into chunks (e.g. 10MB)
        for chunk in response.iter_content(chunk_size=1024 * 1024 * 10):
            f.write(chunk)
    print("Download complete.")
else:
    print("Download failed.")

Download complete.


In [12]:
zip_path = "job_He2COJAL7-results.zip"
extract_to = "."

# Open the zip file in read mode
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

Let's look at what's in the current directory.  We can see a CIF file in `PW p38` (the name I gave to the job), under a directory called `seed_2746317213`.

In [13]:
!ls -R

biotite_intro.ipynb       inference_query_set.json  ref_1bmk.cif
evaluate_cofolding.ipynb  job_He2COJAL7-results.zip summary.txt
experiment_config.json    model_config.json
images                    PW p38

./images:
apheris_lite.png

./PW p38:
seed_2746317213

./PW p38/seed_2746317213:
PW p38_seed_2746317213_sample_1_confidences_aggregated.json
PW p38_seed_2746317213_sample_1_confidences.json
PW p38_seed_2746317213_sample_1_model.cif
timing.json


Use Biotite to read the CIF file into an `atom_array`.

In [14]:
output_filename = "PW p38/seed_2746317213/PW p38_seed_2746317213_sample_1_model.cif"
of3_pdbx_file = pdbx.CIFFile.read(output_filename)
of3_atom_array = pdbx.get_structure(of3_pdbx_file, model=1, include_bonds=True)

Now we will use the `superimpose_chain` function from the [plinder](https://github.com/plinder-org/plinder) package to superimpose two proteins based on a sequence alignment. Note that we copy the aligned protein structure into the variable `of3_atom_array`, replacing the original version.

In [15]:
of3_atom_array, _,_,_ = superimpose_chain(ref_atom_array, of3_atom_array)

Extract the ligand structure from the aligned OpenFold-3 protein

In [16]:
of3_atom_array.chain_id = [x.replace(".1","") for x in of3_atom_array.chain_id]
of3_atom_array.res_name[of3_atom_array.res_name == 'LIG0'] = ligand_ccid

Now let's look at the aligned structures.  We'll define a function that takes a list of `atom_arrays` as input and displays the superimposed structures.

In [17]:
def py3dmol_display(atom_arrays, zoom_to_ligand=True):
    """
    Visualizes structures with robust ligand zooming.
    1. Writes PDBs to viewer.
    2. Identifies 'Interesting' ligands (Not Protein, Not Water).
    3. Zooms specifically to those residues.
    """

    # --- Helper: Write PDB ---
    def array_to_pdb(array):
        f = io.StringIO()
        pdb_file = pdb.PDBFile()
        pdb_file.set_structure(array)
        pdb_file.write(f)
        return f.getvalue()

    palette = [
        ('cyan', 'cyanCarbon'),
        ('orange', 'orangeCarbon'),
        ('magenta', 'magentaCarbon'),
        ('lime', 'greenCarbon'),
        ('white', 'whiteCarbon'),
        ('purple', 'purpleCarbon'),
        ('yellow', 'yellowCarbon')
    ]

    view = py3Dmol.view(width=600, height=400)

    # We will build a list of specific selections for the zoom command
    zoom_selections = []

    for i, array in enumerate(atom_arrays):
        # 1. Add Model
        pdb_str = array_to_pdb(array)
        view.addModel(pdb_str, 'pdb')

        main_color, carbon_scheme = palette[i % len(palette)]

        # 2. Identify The Ligand Atoms using Biotite (Python side)
        #    Criteria: Not Amino Acid AND Not Solvent (Water/Ions)
        #    Note: filter_solvent relies on common names like HOH, WAT, Na, Cl
        ligand_mask = ~struc.filter_amino_acids(array) & ~struc.filter_solvent(array)

        # 3. Apply Styles

        # Protein Style
        view.setStyle({'model': i, 'hetflag': False},
                      {'cartoon': {'color': main_color, 'opacity': 0.85}})

        # Ligand Style (Apply to our calculated mask)
        # We need to extract the residue numbers to create a valid Py3Dmol selection
        if np.any(ligand_mask):
            # Get unique residue numbers (res_id) and chain IDs for the ligand
            ligand_residues = np.unique(array.res_id[ligand_mask])
            ligand_chains = np.unique(array.chain_id[ligand_mask])

            # Convert numpy ints to python ints for JSON serialization
            res_list = [int(r) for r in ligand_residues]
            chain_list = [str(c) for c in ligand_chains]

            # Create a precise selection for this specific ligand
            selection = {
                'model': i,
                'resi': res_list,
                'chain': chain_list
            }

            # Apply stick style to this selection
            view.setStyle(selection,
                          {'stick': {'colorscheme': carbon_scheme, 'radius': 0.3}})

            # Add to zoom list
            zoom_selections.append(selection)

        # (Optional) Handle Solvent/Ions styling if you want them hidden or faint
        # By default, we aren't styling them, so they might show as lines or be hidden depending on defaults.
        # To force hide water:
        # view.setStyle({'model': i, 'resn': 'HOH'}, {'line': {'hidden': True}})

    # 4. Execute Zoom
    if zoom_to_ligand and zoom_selections:
        # 'or' combines the selections so the camera fits ALL ligands in the list
        view.zoomTo({'or': zoom_selections})
    else:
        view.zoomTo()

    return view

With the `py3dmol_display` defined above, we can look at the aligned complexes. The viewer is interactive, hold down the left mouse button or trackpad to rotate the molecule. The mouse wheel or two fingers on the trackpad can be used to zoom.

In [18]:
py3dmol_display([ref_atom_array,of3_atom_array],zoom_to_ligand=True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### Calculate the RMSD Between the X-ray and OpenFold-3 ligands.  
To calculate the ligand RMSD, we must do the following.
- Align the protein structures
- Extract the ligands
- Convert to RDKit molecules
- Calculate the RMSD

We aligned the protein structures above, so we just have to do the three remaining steps.

In [19]:
ref_ligand = ref_atom_array[ref_atom_array.res_name == 'SB5']
of3_ligand = of3_atom_array[of3_atom_array.res_name == 'SB5']
ref_rd_mol = rdkit.to_mol(ref_ligand)
_ = Chem.SanitizeMol(ref_rd_mol)
of3_rd_mol = rdkit.to_mol(of3_ligand)
_ = Chem.SanitizeMol(of3_rd_mol)

In [20]:
rdMolAlign.CalcRMS(ref_rd_mol, of3_rd_mol)

1.3530346001003952

After superimposing the OpenFold-3 and X-ray proteins, an RMSD of 1.35 Å is computed between the `SB5` ligand heavy atoms, and quantifies the pose difference in the aligned binding-site (smaller = closer agreement).  Note the call to [rdMolAlign.CalcRMS](https://www.rdkit.org/docs/source/rdkit.Chem.rdMolAlign.html#rdkit.Chem.rdMolAlign.CalcRMS) considers all symmetry mappings of the two ligands.